<a href="https://colab.research.google.com/drive/1br0hM79ORTVNXUpVgkV5t4o4AigGxfwk?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear Regression from Scratch

In this tutorial, we are going to implement a linear regression model to predict california housing prices. We will build the model **from scratch** using numpy. This will be a great approach to begin understanding regression based models.

After completing this tutorial the learner is expected to know the basic building blocks of a linear regression model. The learner is also expected to know the **pipeline** of reading and transforming data for machine learning **workflows**.



In [ ]:
## Import the usual libraries 
import numpy as np # 导包
import pandas as pd # 导包
from sklearn.datasets import fetch_california_housing # 导包，加州房价数据集
from sklearn.model_selection import train_test_split # 导包
from sklearn.preprocessing import StandardScaler # 导包
import matplotlib.pyplot as plt # 导包

# Importing the dataset

The real-world dataset can be obtained by the function `fetch_california_housing` that downloads the dataset for us. 

The `as_frame` parameter returns a pandas dataframe which is a library useful for viewing contents of the data.



In [ ]:
# Fetch the data using sklearn function
bunch = fetch_california_housing(download_if_missing=True, as_frame=True) # 加载数据集，并在数据集中生成pandas的数据表格

# Load the dataframe and view
df = bunch.frame # 获取数据表格
df.head() # 返回表格的前几行，参数n，默认5

For this dataset, our target variable is **the median house value** for California **districts**, **expressed** in hundreds of thousands of dollars ($100,000).

We can take a **closer** look at the various statistical parameters of the dataset using pandas. The `describe` function will help us.

In [ ]:
df.describe() # 生成描述性统计数据，不包括NaN值

As we can see the data in each of the columns is on different **scales**. For example, the average bedroom value is around 1 and the average population is around 1425. 

Generally, machine learing models do not work well when the data is on different scales. Thus, we have to **normalize** our data in the range [-1,1]. The module [StandardScalar](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) will help us in this.

The training data should always be normalized. The testing data should be normalized using the values of the training data. 

In [ ]:
# !wget https://raw.githubusercontent.com/Ankit152/Fish-Market/main/Fish.csv
# import pandas as pd
# df  = pd.read_csv("Fish.csv")
# y = df['Weight']
# x = df[["Length1", "Length2", "Length3", "Height", "Width","Weight"]]

df = bunch.frame
x = df.iloc[:,:-1] # Select all the columns, except the last column 分割出特征
y = df.iloc[:,-1:] # Select the last column # 分割出label
# random_state随机数种子：其实就是该组随机数的编号，在需要重复试验的时候，保证得到一组一样的随机数。比如你每次都填1，其他参数一样的情况下你得到的随机数组是一样的。但填0或不填，每次都会不一样。
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.33, random_state = 1) # 分割数据集，测试集占33%

input_scalar = StandardScaler() # 输入归一化，默认使用均值u和标准差σ，x_* = (x-u)/σ
output_scalar = StandardScaler() # 输出归一化

x_train = input_scalar.fit_transform(x_train).T # Normalize train data。fit_transform进行数据归一化，并且求转置，shape从(n_features, m_samples)转为(m_samples, n_features)。xx.T再求转置
x_test = input_scalar.transform(x_test).T # Only transform test data using values of train data。同理

y_train = output_scalar.fit_transform(y_train).reshape(-1) # 同理。reshape(-1)转成一维数组
y_test = output_scalar.transform(y_test).reshape(-1) # 同理

dataset_copy = [ x_train.copy(), x_test.copy(),  y_train.copy(),  y_test.copy()] # 深拷贝一份数据集

# Linear Regression Model

Now we define our linear regression model from scratch.

A linear regression model is of the form:

$y = a_1 x_1 + a_2 x_2 + \dots + a_nx_n + a_{n+1}$
  
The above can be rewritten using matrix multiplication as

$ y = w^T x $

where 

$ w = [a_1, a_2, \dots,  a_n, a_{n+1}]^T $

$ x = [x_1, x_2, \dots,  x_n]^T $


In [ ]:
class LinearRegression(): # 线性回归类
  def __init__(self, dim, lr = 0.1): # 初始化函数，学习率默认0.1
    assert isinstance
    self.lr = lr # 初始化学习率
    self.w = np.zeros((dim)) # 初始化权重
    self.grads = {"dw": np.zeros((dim)) +5} # 初始化梯度

  def forward(self, x):
    y = self.w @ x # numpy中的矩阵乘法，与dot()一样，这里w无需转置，转置后还是一维数组
    return y # 返回预测结果，一维数组，用w对x的每一列做点积
  
  def backward(self, x, y_hat, y):
    assert y_hat.shape == y.shape # 判断预测和真实y形状是否相同，若不同，抛异常AssertionError
    self.grads["dw"] = (1 / x.shape[1]) * ((y_hat - y) @ x.T).T # 计算梯度，就是对代价函数求偏导数，这个公式就是对均方误差求偏导数的结果
    assert self.grads["dw"].shape == self.w.shape # 判断形状
    
    # print(self.grads["dw"])

  def optimize(self):
    self.w = self.w - self.lr * self.grads["dw"] # 优化器，梯度下降，调整参数w

# Loss

For linear regression, various loss functions such as the mean absolute error, mean squared error, or root mean squared error can be used.

In this example, we will use the mean squared error (MSE) loss.

The MSE loss is given by 

$ error = \frac{1}{m} Σ_{i=1}^{m} (y_{true}^{i} - y_{pred}^{i})^2 $ 

where $i$ denotes the particular obseration/row in the dataset and $m$ is the total number of obserations.

To ensure our model is correct, the loss should decrease over each epoch.


In [ ]:
num_epochs = 10000 # 对训练集训练10000次
train_loss_history = [] # 训练损失记录
test_loss_history = [] # 测试损失记录
w_history = [] # 权重记录
dim = x_train.shape[0] # 维度（几个特征）
num_train = x_train.shape[1] # 训练集长度
num_test = x_test.shape[1] # 测试集长度


model = LinearRegression(dim = dim, lr = 0.1) # 实例化模型
for i in range(num_epochs): # 循环10000次
  y_hat = model.forward(x_train) # 计算预测值
  train_loss = 1/(2 * num_train) * ((y_train - y_hat) ** 2).sum() # 均方误差

  w_history.append(model.w) # 记录这个epoch的权重
  model.backward(x_train,y_hat,y_train) # 反向计算梯度
  model.optimize() # 优化参数

  y_hat = model.forward(x_test) # 利用测试集预测
  test_loss = 1/(2 * num_test) * ((y_test - y_hat) ** 2).sum() # 损失

  train_loss_history.append(train_loss) # 记录训练集损失
  test_loss_history.append(test_loss) # 记录测试集损失

  if i % 20 == 0:
    print(f"Epoch {i} | Train Loss {train_loss} | Test Loss {test_loss}") # 打印当前epoch两个损失

plt.plot(range(num_epochs), train_loss_history, label = "Training") # 画训练损失曲线
plt.plot(range(num_epochs), test_loss_history, label = "Test") # 画测试集损失曲线
plt.legend() # 生成
plt.show() # 展示

# Results

Before viewing the results, we need to reverse the transformations applied on the output variable y.

The `inverse_transform` method of the StandardScaler object will help us.

In [ ]:
from sklearn.metrics import mean_squared_error
y_test = output_scalar.inverse_transform(y_test[np.newaxis,:]) # 反归一化
y_hat  = output_scalar.inverse_transform(y_hat[np.newaxis,:]) # 反归一化
error = (((y_test - y_hat) ** 2).sum() / num_test ) # 计算均方误差
print("Test Set Error", error) # 打印

# Libraries

Instead of coding everything from scratch, i.e the model, loss functions, and gradient calculations, there are many libaries that have implemented many machine learning algorithms for us.

These libraries will generally be faster and more optimized. We can use the LinearRegression and SGD regressor module from scikit learn to compare our model

In [ ]:
from sklearn.linear_model import SGDRegressor # 导包，随机梯度下降的线性回归


x_train, x_test, y_train, y_test = dataset_copy # 训练集和测试集
sgd = SGDRegressor() # 实例化对象
sgd.fit(x_train.T, y_train) # 训练
y_hat = sgd.predict(x_test.T) # 预测
y_test = output_scalar.inverse_transform(y_test[np.newaxis,:]) # 同理
y_hat  = output_scalar.inverse_transform(y_hat[np.newaxis,:]) # 同理
error = mean_squared_error(y_test, y_hat, squared = True) # 计算均方误差 squared默认True，返回均方误差，如果是False返回均方误差的平方根
print("Test Set Error", error) # 打印

In [ ]:
from sklearn.linear_model import LinearRegression as LR # 导包，线性回归

x_train, x_test, y_train, y_test = dataset_copy # 数据集
lr = LR() # 实例化
lr.fit(x_train.T, y_train) # 训练
y_hat = lr.predict(x_test.T) # 预测
y_test = output_scalar.inverse_transform(y_test[np.newaxis,:]) # 同理
y_hat  = output_scalar.inverse_transform(y_hat[np.newaxis,:]) # 同理
error = mean_squared_error(y_test, y_hat, squared = True) # 同理
print("Test Set Error", error) # 打印